In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import os

spark = SparkSession.builder.master("local").appName("Test").getOrCreate()


In [2]:
movies_metadata_path = '../input/movies_metadata.csv'
df_movies = spark.read.csv(movies_metadata_path, header=True)


In [3]:
df_geral = (
  df_movies
  .withColumn("imdbId", F.substring(F.col("imdb_id"), 3, 1000))# tirando tt
  .select("imdbId","popularity")
  .withColumn('popularity', F.log1p(F.col('popularity').cast("float")))
  .dropna()
)

In [4]:
df_top_movies = (
  df_geral
  .withColumn("top_movies", F.round(F.when(F.col("popularity")> 3, F.lit(1)).otherwise(F.col("popularity")/3),4))
  .filter(F.col('top_movies') == 1)
  .select("imdbId")
)

In [5]:
df_top_movies.show(truncate=False)

+-------+
|imdbId |
+-------+
|0114709|
|0112573|
|0076759|
|0110912|
|0111161|
|0109830|
|0110357|
|0108052|
|0083658|
|0103064|
|0096895|
|0101414|
|0116136|
|0117951|
|0068646|
|0032138|
|0062622|
|0083866|
|0092099|
|0073486|
+-------+
only showing top 20 rows



In [9]:
geracao_path = os.path.join('..','geracao_candidatos')
df_top_movies.repartition(5).write.mode('overwrite').parquet(geracao_path)